In [1]:
import os
import numpy as np
import pandas as pd
import scipy
import anndata
import scanpy as sc
import seaborn as sns
import scipy.sparse as sp
import matplotlib.pyplot as plt

# NOTES TO SELF
You can skip to "Load Result X" cell if that step has already been completed

# load metadata
**Note: will have to move some files around to use the commands given in F1L**

In [2]:
cwd = os.getcwd()
meta = pd.read_csv(cwd+'/data/Metadata.txt', sep='\t',)
meta.drop([0], axis=0, inplace=True)
meta.rename(columns={'NAME':'CellID', 'Cell_line':'CellLine', 'Pool_ID':'Pool', 'Cancer_type':'Indication'}, inplace=True)
meta

/tmp/ipykernel_11933/565737092.py:2: DtypeWarning: Columns (4,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv(cwd+'/data/Metadata.txt', sep='\t',)


,CellID,CellLine,Pool,Indication,Genes_expressed,Discrete_cluster_minpts5_eps1.8,Discrete_cluster_minpts5_eps1.5,Discrete_cluster_minpts5_eps1.2,CNA_subclone,SkinPig_score,...,EMTII_score,EMTIII_score,IFNResp_score,p53Sen_score,EpiSen_score,StressResp_score,ProtMatu_score,ProtDegra_score,G1/S_score,G2/M_score
1,AAACCTGAGACATAAC-1-18,NCIH2126_LUNG,18,Lung Cancer,4318,NaN,NaN,NaN,NaN,0.166,...,-0.935,-0.935,0.13,0.619,1.869,-0.004,0.805,0.896,0.424,-1.125
2,AACGTTGTCACCCGAG-1-18,NCIH2126_LUNG,18,Lung Cancer,5200,NaN,NaN,NaN,NaN,-0.213,...,-1.027,-1.027,0.066,1.049,1.267,0.252,1.299,1.61,0.624,-0.048
3,AACTGGTAGACACGAC-1-18,NCIH2126_LUNG,18,Lung Cancer,4004,NaN,NaN,NaN,NaN,-0.101,...,-0.677,-0.677,0.304,0.822,2.401,0.141,0.451,1.225,-0.795,0.064
4,AACTGGTAGGGCTTGA-1-18,NCIH2126_LUNG,18,Lung Cancer,4295,NaN,NaN,NaN,NaN,-0.014,...,-0.735,-0.735,0.094,0.834,2.282,0.15,0.267,0.892,-0.238,1.118
5,AACTGGTAGTACTTGC-1-18,NCIH2126_LUNG,18,Lung Cancer,4842,NaN,NaN,NaN,NaN,0.006,...,-0.821,-0.821,0.034,0.96,1.4,-0.012,-0.276,-0.428,0.267,0.791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53509,c4722,JHU006_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,3343,NaN,NaN,NaN,NaN,0.018,...,-0.505,-0.505,1.657,1.583,3.85,0.539,0.473,0.544,-1.079,-1.349
53510,c4724,JHU006_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,6977,NaN,NaN,NaN,NaN,-0.098,...,-0.876,-0.876,0.669,1.086,3.046,0.799,0.49,1.319,-0.37,0.057
53511,c4731,JHU006_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,6638,NaN,NaN,NaN,NaN,-0.112,...,-0.112,-0.112,0.61,0.693,2.289,0.65,0.729,1.143,-0.508,0.501
53512,c4735,JHU006_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,4052,NaN,NaN,NaN,NaN,-0.244,...,1.981,1.981,0.523,-0.309,0.267,0.822,1.049,0.777,0.296,-0.936


In [3]:
meta.dtypes

CellID                             object
CellLine                           object
Pool                               object
Indication                         object
Genes_expressed                    object
Discrete_cluster_minpts5_eps1.8    object
Discrete_cluster_minpts5_eps1.5    object
Discrete_cluster_minpts5_eps1.2    object
CNA_subclone                       object
SkinPig_score                      object
EMTI_score                         object
EMTII_score                        object
EMTIII_score                       object
IFNResp_score                      object
p53Sen_score                       object
EpiSen_score                       object
StressResp_score                   object
ProtMatu_score                     object
ProtDegra_score                    object
G1/S_score                         object
G2/M_score                         object
dtype: object

In [4]:
# # I THOUGHT THAT "Discrete_cluster_minpts5_eps1.8" were numeric values, but turns out
# # they are strings. i leave this here as a reminder

# display(meta.loc[:,"Discrete_cluster_minpts5_eps1.8"])
# np.array(meta.loc[:,"Discrete_cluster_minpts5_eps1.8"],dtype=np.float32)
# # print("number NaN = ",sum(np.isnan()))

# # plt.plot(meta.loc[:,"Discrete_cluster_minpts5_eps1.8"],bins=10, edgecolor='k', alpha=0.7)

In [5]:
# Iterate through the columns and change their type
for col in meta.columns:
    lowercase_name = col.lower() # get name (case insensitive)
    if "score" in lowercase_name:  # Check if 'score' is in the column name 
        meta[col] = meta[col].astype(np.float32)
    elif "expressed" in lowercase_name:
        meta[col] = meta[col].astype(np.int32) 
    else:
        meta[col] = meta[col].astype(str)

print(meta.dtypes)

CellID                              object
CellLine                            object
Pool                                object
Indication                          object
Genes_expressed                      int32
Discrete_cluster_minpts5_eps1.8     object
Discrete_cluster_minpts5_eps1.5     object
Discrete_cluster_minpts5_eps1.2     object
CNA_subclone                        object
SkinPig_score                      float32
EMTI_score                         float32
EMTII_score                        float32
EMTIII_score                       float32
IFNResp_score                      float32
p53Sen_score                       float32
EpiSen_score                       float32
StressResp_score                   float32
ProtMatu_score                     float32
ProtDegra_score                    float32
G1/S_score                         float32
G2/M_score                         float32
dtype: object


# load cell ids
because large amount of data, we only do the first 10 rows to see what we're working with

In [6]:
# the first three rows are CellID, CellLine, and Pool
# only need the first row, CellID
example = pd.read_csv(cwd+'/data/UMIcount_data.txt', nrows=10, sep='\t', header=None)
display(example)

,0,1,2,3,4,5,6,7,8,9,...,56973,56974,56975,56976,56977,56978,56979,56980,56981,56982
0,NaN,AAACCTGAGACATAAC-1-18,AAACCTGCACAACGCC-1-18,AAACCTGCAGACAAGC-1-18,AAACCTGCAGCTCGAC-1-18,AAACCTGCATGGATGG-1-18,AAACCTGGTACGAAAT-1-18,AAACGGGAGATACACA-1-18,AAACGGGCAGGTGCCT-1-18,AAACGGGGTTTAGGAA-1-18,...,c4781,c4784,c4785,c4786,c4787,c4788,c4789,c4793,c4800,c4812
1,Cell_line,NCIH2126_LUNG,SW579_THYROID,C32_SKIN,SW579_THYROID,NCIH446_LUNG,HEC251_ENDOMETRIUM,MFE319_ENDOMETRIUM,SKNAS_AUTONOMIC_GANGLIA,NCIH2452_PLEURA,...,SCC25_UPPER_AERODIGESTIVE_TRACT,SCC25_UPPER_AERODIGESTIVE_TRACT,SCC25_UPPER_AERODIGESTIVE_TRACT,SCC25_UPPER_AERODIGESTIVE_TRACT,93VU_UPPER_AERODIGESTIVE_TRACT,JHU029_UPPER_AERODIGESTIVE_TRACT,SCC9_UPPER_AERODIGESTIVE_TRACT,JHU029_UPPER_AERODIGESTIVE_TRACT,SCC9_UPPER_AERODIGESTIVE_TRACT,SCC9_UPPER_AERODIGESTIVE_TRACT
2,Pool_ID,18,18,18,18,18,18,18,18,18,...,custom,custom,custom,custom,custom,custom,custom,custom,custom,custom
3,A1BG,0,1,1,1,0,2,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,A1BG-AS1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,A1CF,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,A2M,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,A2M-AS1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,A2ML1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,A2ML1-AS1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
counts_cellid = pd.read_csv(cwd+'/data/UMIcount_data.txt', nrows=1, sep='\t', header=None) # get counts_cellid dataframe, 1x56983
counts_cellid = counts_cellid.transpose() # transpose counts_cellid dataframe to 56983x1

print(counts_cellid.iloc[0])

# drop the NaN with label 0, see print out from line above and also the previous cell. 
# counts_cellid dataframe will be 56982x1
counts_cellid.drop([0], inplace=True) 
counts_cellid

0    NaN
Name: 0, dtype: object


,0
1,AAACCTGAGACATAAC-1-18
2,AAACCTGCACAACGCC-1-18
3,AAACCTGCAGACAAGC-1-18
4,AAACCTGCAGCTCGAC-1-18
5,AAACCTGCATGGATGG-1-18
...,...
56978,c4788
56979,c4789
56980,c4793
56981,c4800


# load full count data
NOTE: if the file 'raw_adata_56982by30314.h5ad' exists, skip the next two subheadings and go straight to loading that adata object

In [8]:
if os.path.exists('data/raw_adata_56982by30314.h5ad'):
    print('Already created this object, don\'t need to recreate it!')

Already created this object, don't need to recreate it!


## get counts matrix in sparse array format

In [9]:
example.shape[1] # check we can get the exact number of cell IDs as above without hardcoding

56983

In [10]:
%%time

# APPROACH 1: DIDN'T WORK, BREAKS LAPTOP
# # we skip first 3 rows based on what we saw above when we printed the first 10 rows, since
# # first 3 rows are CellID, CellLine, and pool
# counts = pd.read_csv(cwd+'/data/UMIcount_data.txt', sep='\t', skiprows=3, header=None, index_col=0)
# counts = counts.transpose()
# counts

# APPROACH 2: DIDN'T WORK, NEED TO SKIP THINGS
# anndata.io.read_csv(filename, delimiter=',', first_column_names=None, dtype='float32')
# anndata.read_csv(cwd+'/data/UMIcount_data.txt', delimiter='\t') # NOT ENOUGH OPTIONS TO SKIP ROWS OR COLS

# APPROACH 3: STARTED 8:27pm-8:29pm
counts = np.loadtxt(
    'data/UMIcount_data.txt', 
    delimiter='\t', 
    skiprows=3,
    usecols=range(1,example.shape[1]) # skip first col, this is gene IDs
    )

CPU times: user 2min 46s, sys: 9.1 s, total: 2min 55s
Wall time: 2min 55s


In [11]:
%%time
counts = counts.T

CPU times: user 10 μs, sys: 0 ns, total: 10 μs
Wall time: 12.6 μs


In [12]:
counts.shape

(56982, 30314)

In [13]:
counts

array([[0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 2., 0.]])

In [14]:
%%time
counts = sp.csr_matrix(counts)

CPU times: user 2min 21s, sys: 6.94 s, total: 2min 28s
Wall time: 2min 28s


## get gene ids

In [15]:
%%time

# the first column is the gene IDs after skipping first 3 rows
gene_ids = pd.read_csv(cwd+'/data/UMIcount_data.txt', sep='\t', 
                       skiprows=3, header=None, usecols=[0])

CPU times: user 36.7 s, sys: 1.06 s, total: 37.7 s
Wall time: 37.5 s


In [16]:
gene_ids.shape

(30314, 1)

In [17]:
gene_ids[0]

0                  A1BG
1              A1BG-AS1
2                  A1CF
3                   A2M
4               A2M-AS1
              ...      
30309           SPATA13
30310           TBC1D26
30311           TIMM10B
30312            TMBIM4
30313    TMEM256-PLSCR3
Name: 0, Length: 30314, dtype: object

## get gene_ids and counts matrix into AnnData, and save

In [18]:
%%time
raw_adata = anndata.AnnData(X=counts)
print('good 1')
raw_adata.var_names = gene_ids[0]
print('good 2')
raw_adata.write('data/raw_adata_56982by30314.h5ad')

good 1
good 2
CPU times: user 932 ms, sys: 4.01 s, total: 4.94 s
Wall time: 7.39 s


In [19]:
try:
    del counts
except:
    print('counts already deleted')

### Load result cell 1
Raw adata, still has extra cells which don't have CellIDs in metadata (QC sequences?)

In [20]:
%%time
raw_adata = anndata.read_h5ad('data/raw_adata_56982by30314.h5ad')

CPU times: user 771 ms, sys: 1.93 s, total: 2.7 s
Wall time: 2.5 s


## Drop CellIDs (obs) that are NOT in meta['CellID'] column
SECTION COMPLETED, SKIP TO "LOAD RESULT CELL 2"

In [21]:
try:
    filtered_adata = raw_adata.copy()
    del raw_adata
except:
    print('raw_adata already deleted, and filtered created')
    print('filtered_adata:',filtered_adata)

In [22]:
display(counts_cellid)

,0
1,AAACCTGAGACATAAC-1-18
2,AAACCTGCACAACGCC-1-18
3,AAACCTGCAGACAAGC-1-18
4,AAACCTGCAGCTCGAC-1-18
5,AAACCTGCATGGATGG-1-18
...,...
56978,c4788
56979,c4789
56980,c4793
56981,c4800


In [23]:
counts_index = counts_cellid[0] # the collection of Cell_IDs is the desired index for the counts matrix
mask = counts_index.isin(meta['CellID'])

In [24]:
for i, truth_cell_id in enumerate(mask):
    if not truth_cell_id:
        print(i, filtered_adata.obs_names[i])

28 28
52 52
279 279
290 290
418 418
440 440
459 459
560 560
572 572
585 585
604 604
661 661
665 665
698 698
755 755
807 807
815 815
847 847
849 849
867 867
935 935
984 984
1067 1067
1141 1141
1200 1200
1260 1260
1301 1301
1323 1323
1326 1326
1341 1341
1403 1403
1476 1476
1704 1704
1735 1735
1897 1897
1901 1901
1927 1927
1985 1985
2075 2075
2090 2090
2121 2121
2250 2250
2266 2266
2391 2391
2485 2485
2507 2507
2520 2520
2613 2613
2624 2624
2697 2697
2787 2787
2855 2855
2859 2859
2899 2899
2919 2919
2920 2920
2924 2924
2983 2983
3019 3019
3043 3043
3110 3110
3137 3137
3229 3229
3241 3241
3398 3398
3429 3429
3445 3445
3495 3495
3523 3523
3637 3637
3661 3661
3713 3713
3754 3754
3822 3822
3873 3873
3913 3913
3946 3946
4005 4005
4048 4048
4054 4054
4084 4084
4112 4112
4270 4270
4320 4320
4339 4339
4363 4363
4383 4383
4391 4391
4451 4451
4466 4466
4476 4476
4495 4495
4506 4506
4532 4532
4572 4572
4649 4649
4896 4896
4964 4964
5048 5048
5614 5614
5637 5637
5657 5657
5899 5899
5931 5931
6021 602

In [25]:
filtered_adata.obs_names[mask]

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '56972', '56973', '56974', '56975', '56976', '56977', '56978', '56979',
       '56980', '56981'],
      dtype='object', length=53513)

In [26]:
meta.index

RangeIndex(start=1, stop=53514, step=1)

In [27]:
sum(mask) # sum counts number of "True" in boolean array, matches original notebook value!

53513

In [28]:
sum(~mask)

3469

In [29]:
filtered_adata.X[~mask]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 7091798 stored elements and shape (3469, 30314)>

In [30]:
meta.shape

(53513, 21)

In [31]:
mask.shape

(56982,)

In [32]:
print(filtered_adata.X[mask].shape,len(gene_ids))

(53513, 30314) 30314


In [33]:
meta_new = meta.set_index('CellID',drop=True,inplace=False) # we need CellID to be the index, create a separate meta object with CellID as index
meta_new

,CellLine,Pool,Indication,Genes_expressed,Discrete_cluster_minpts5_eps1.8,Discrete_cluster_minpts5_eps1.5,Discrete_cluster_minpts5_eps1.2,CNA_subclone,SkinPig_score,EMTI_score,EMTII_score,EMTIII_score,IFNResp_score,p53Sen_score,EpiSen_score,StressResp_score,ProtMatu_score,ProtDegra_score,G1/S_score,G2/M_score
CellID,,,,,,,,,,,,,,,,,,,,
AAACCTGAGACATAAC-1-18,NCIH2126_LUNG,18,Lung Cancer,4318,nan,nan,nan,nan,0.166,-0.045,-0.935,-0.935,0.130,0.619,1.869,-0.004,0.805,0.896,0.424,-1.125
AACGTTGTCACCCGAG-1-18,NCIH2126_LUNG,18,Lung Cancer,5200,nan,nan,nan,nan,-0.213,0.035,-1.027,-1.027,0.066,1.049,1.267,0.252,1.299,1.610,0.624,-0.048
AACTGGTAGACACGAC-1-18,NCIH2126_LUNG,18,Lung Cancer,4004,nan,nan,nan,nan,-0.101,-0.183,-0.677,-0.677,0.304,0.822,2.401,0.141,0.451,1.225,-0.795,0.064
AACTGGTAGGGCTTGA-1-18,NCIH2126_LUNG,18,Lung Cancer,4295,nan,nan,nan,nan,-0.014,-0.093,-0.735,-0.735,0.094,0.834,2.282,0.150,0.267,0.892,-0.238,1.118
AACTGGTAGTACTTGC-1-18,NCIH2126_LUNG,18,Lung Cancer,4842,nan,nan,nan,nan,0.006,-0.055,-0.821,-0.821,0.034,0.960,1.400,-0.012,-0.276,-0.428,0.267,0.791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c4722,JHU006_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,3343,nan,nan,nan,nan,0.018,-0.149,-0.505,-0.505,1.657,1.583,3.850,0.539,0.473,0.544,-1.079,-1.349
c4724,JHU006_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,6977,nan,nan,nan,nan,-0.098,-0.197,-0.876,-0.876,0.669,1.086,3.046,0.799,0.490,1.319,-0.370,0.057
c4731,JHU006_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,6638,nan,nan,nan,nan,-0.112,-0.107,-0.112,-0.112,0.610,0.693,2.289,0.650,0.729,1.143,-0.508,0.501


In [34]:
meta_new = meta_new.reindex(index=counts_cellid[0].tolist()).dropna(how='all')
# this will do a couple things:
# (1) meta_new.reindex: arrange meta_new rows in the order of counts_index
# (2) .dropna(how='all'): delete meta_new rows whose index are not found in counts_index (whose rows filled with nan by default by reindex)
# (3) create a copy of meta_new meeting the above (by default) and assign to meta_new
meta_new

,CellLine,Pool,Indication,Genes_expressed,Discrete_cluster_minpts5_eps1.8,Discrete_cluster_minpts5_eps1.5,Discrete_cluster_minpts5_eps1.2,CNA_subclone,SkinPig_score,EMTI_score,EMTII_score,EMTIII_score,IFNResp_score,p53Sen_score,EpiSen_score,StressResp_score,ProtMatu_score,ProtDegra_score,G1/S_score,G2/M_score
CellID,,,,,,,,,,,,,,,,,,,,
AAACCTGAGACATAAC-1-18,NCIH2126_LUNG,18,Lung Cancer,4318.0,nan,nan,nan,nan,0.166,-0.045,-0.935,-0.935,0.130,0.619,1.869,-0.004,0.805,0.896,0.424,-1.125
AAACCTGCACAACGCC-1-18,SW579_THYROID,18,Thyroid Cancer,5021.0,nan,nan,SW579_THYROID_1,nan,-0.056,0.776,0.953,0.953,-0.266,-0.334,-1.125,-0.039,-0.243,-0.642,-0.173,1.365
AAACCTGCAGACAAGC-1-18,C32_SKIN,18,Skin Cancer,3047.0,nan,nan,nan,nan,1.092,0.617,-0.034,-0.034,0.318,0.570,-0.165,0.074,0.250,0.096,-0.367,-1.135
AAACCTGCAGCTCGAC-1-18,SW579_THYROID,18,Thyroid Cancer,2765.0,nan,nan,SW579_THYROID_1,nan,-0.601,1.038,1.952,1.952,0.341,-0.253,-0.552,0.921,2.876,1.645,0.226,0.469
AAACCTGCATGGATGG-1-18,NCIH446_LUNG,18,Lung Cancer,2064.0,nan,nan,nan,nan,-0.251,-0.325,0.258,0.258,-0.044,-1.256,-0.367,-0.317,0.790,1.925,0.138,-0.384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c4788,JHU029_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,5929.0,JHU029_UPPER_AERODIGESTIVE_TRACT_1,JHU029_UPPER_AERODIGESTIVE_TRACT_1,JHU029_UPPER_AERODIGESTIVE_TRACT_1,nan,-0.317,-0.390,0.023,0.023,-0.100,-0.604,-0.358,-0.123,0.067,0.804,0.135,0.264
c4789,SCC9_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,3531.0,SCC9_UPPER_AERODIGESTIVE_TRACT_2,SCC9_UPPER_AERODIGESTIVE_TRACT_2,SCC9_UPPER_AERODIGESTIVE_TRACT_2,SCC9_UPPER_AERODIGESTIVE_TRACT_2,0.238,1.176,2.094,2.094,0.014,-0.045,-0.050,0.214,0.238,-0.514,-1.021,-1.243
c4793,JHU029_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,4029.0,JHU029_UPPER_AERODIGESTIVE_TRACT_1,JHU029_UPPER_AERODIGESTIVE_TRACT_3,nan,nan,-0.164,-0.163,-0.405,-0.405,0.010,-0.008,0.045,0.265,0.054,-0.466,0.345,0.804


In [35]:
print(filtered_adata.X[mask].shape,'\n',
      meta_new.shape)

(53513, 30314) 
 (53513, 20)


In [36]:
%%time

# Instatiate the AnnData object with all the valid cell id's found in meta data
filtered_adata = anndata.AnnData(X = filtered_adata.X[mask],
                        var=gene_ids,
                        obs=meta_new)

CPU times: user 719 ms, sys: 698 ms, total: 1.42 s
Wall time: 1.28 s


/home/ethan_unix_0/anaconda3/envs/2024-F1L/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [37]:
display(filtered_adata.obs,filtered_adata.var)

,CellLine,Pool,Indication,Genes_expressed,Discrete_cluster_minpts5_eps1.8,Discrete_cluster_minpts5_eps1.5,Discrete_cluster_minpts5_eps1.2,CNA_subclone,SkinPig_score,EMTI_score,EMTII_score,EMTIII_score,IFNResp_score,p53Sen_score,EpiSen_score,StressResp_score,ProtMatu_score,ProtDegra_score,G1/S_score,G2/M_score
CellID,,,,,,,,,,,,,,,,,,,,
AAACCTGAGACATAAC-1-18,NCIH2126_LUNG,18,Lung Cancer,4318.0,nan,nan,nan,nan,0.166,-0.045,-0.935,-0.935,0.130,0.619,1.869,-0.004,0.805,0.896,0.424,-1.125
AAACCTGCACAACGCC-1-18,SW579_THYROID,18,Thyroid Cancer,5021.0,nan,nan,SW579_THYROID_1,nan,-0.056,0.776,0.953,0.953,-0.266,-0.334,-1.125,-0.039,-0.243,-0.642,-0.173,1.365
AAACCTGCAGACAAGC-1-18,C32_SKIN,18,Skin Cancer,3047.0,nan,nan,nan,nan,1.092,0.617,-0.034,-0.034,0.318,0.570,-0.165,0.074,0.250,0.096,-0.367,-1.135
AAACCTGCAGCTCGAC-1-18,SW579_THYROID,18,Thyroid Cancer,2765.0,nan,nan,SW579_THYROID_1,nan,-0.601,1.038,1.952,1.952,0.341,-0.253,-0.552,0.921,2.876,1.645,0.226,0.469
AAACCTGCATGGATGG-1-18,NCIH446_LUNG,18,Lung Cancer,2064.0,nan,nan,nan,nan,-0.251,-0.325,0.258,0.258,-0.044,-1.256,-0.367,-0.317,0.790,1.925,0.138,-0.384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c4788,JHU029_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,5929.0,JHU029_UPPER_AERODIGESTIVE_TRACT_1,JHU029_UPPER_AERODIGESTIVE_TRACT_1,JHU029_UPPER_AERODIGESTIVE_TRACT_1,nan,-0.317,-0.390,0.023,0.023,-0.100,-0.604,-0.358,-0.123,0.067,0.804,0.135,0.264
c4789,SCC9_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,3531.0,SCC9_UPPER_AERODIGESTIVE_TRACT_2,SCC9_UPPER_AERODIGESTIVE_TRACT_2,SCC9_UPPER_AERODIGESTIVE_TRACT_2,SCC9_UPPER_AERODIGESTIVE_TRACT_2,0.238,1.176,2.094,2.094,0.014,-0.045,-0.050,0.214,0.238,-0.514,-1.021,-1.243
c4793,JHU029_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,4029.0,JHU029_UPPER_AERODIGESTIVE_TRACT_1,JHU029_UPPER_AERODIGESTIVE_TRACT_3,nan,nan,-0.164,-0.163,-0.405,-0.405,0.010,-0.008,0.045,0.265,0.054,-0.466,0.345,0.804


,0
0,A1BG
1,A1BG-AS1
2,A1CF
3,A2M
4,A2M-AS1
...,...
30309,SPATA13
30310,TBC1D26
30311,TIMM10B
30312,TMBIM4


In [38]:
# rename the vars column from "0" to "GeneID"
filtered_adata.var.rename(columns={0:'GeneID'}, inplace=True)
display(filtered_adata.var)

# adata.obs.rename(columns={0:'CellID'}, inplace=True)
# display(adata.obs,adata.var)

,GeneID
0,A1BG
1,A1BG-AS1
2,A1CF
3,A2M
4,A2M-AS1
...,...
30309,SPATA13
30310,TBC1D26
30311,TIMM10B
30312,TMBIM4


In [39]:
%%time 
# save the result
filtered_adata.write('data/filtered_adata_53513by30314.h5ad')

CPU times: user 1.42 s, sys: 4.73 s, total: 6.15 s
Wall time: 5.73 s


### Load result cell 2
Filtered so only cells from the metadata are in the anndata object

In [40]:
%%time
# load the result
filtered_adata = anndata.read_h5ad('data/filtered_adata_53513by30314.h5ad')

CPU times: user 676 ms, sys: 2.24 s, total: 2.92 s
Wall time: 2.68 s


In [41]:
print(filtered_adata.obs_names.equals(meta_new.index))  # Should return True

True


# verify agrees with original notebook
check with [original notebook](https://github.com/deanslee/FigureOneLab/blob/main/kinker/240701_kinker_anndata.ipynb) that the adata object looks correct

In [42]:
filtered_adata

AnnData object with n_obs × n_vars = 53513 × 30314
    obs: 'CellLine', 'Pool', 'Indication', 'Genes_expressed', 'Discrete_cluster_minpts5_eps1.8', 'Discrete_cluster_minpts5_eps1.5', 'Discrete_cluster_minpts5_eps1.2', 'CNA_subclone', 'SkinPig_score', 'EMTI_score', 'EMTII_score', 'EMTIII_score', 'IFNResp_score', 'p53Sen_score', 'EpiSen_score', 'StressResp_score', 'ProtMatu_score', 'ProtDegra_score', 'G1/S_score', 'G2/M_score'
    var: 'GeneID'

In [43]:
filtered_adata.X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 201078579 stored elements and shape (53513, 30314)>

In [48]:
53513*30314

1622193082

In [44]:
201078579/(53513*30314) # shows we are saving lots of space by not storing zeroes; we are only storing 12% of the matrix!

0.12395477531693727

In [45]:
%%time
sc.pp.filter_genes(filtered_adata, min_cells=10)
sc.pp.filter_cells(filtered_adata, min_genes=200)

CPU times: user 7.09 s, sys: 7.14 s, total: 14.2 s
Wall time: 16.4 s


In [46]:
filtered_adata

AnnData object with n_obs × n_vars = 53513 × 23081
    obs: 'CellLine', 'Pool', 'Indication', 'Genes_expressed', 'Discrete_cluster_minpts5_eps1.8', 'Discrete_cluster_minpts5_eps1.5', 'Discrete_cluster_minpts5_eps1.2', 'CNA_subclone', 'SkinPig_score', 'EMTI_score', 'EMTII_score', 'EMTIII_score', 'IFNResp_score', 'p53Sen_score', 'EpiSen_score', 'StressResp_score', 'ProtMatu_score', 'ProtDegra_score', 'G1/S_score', 'G2/M_score', 'n_genes'
    var: 'GeneID', 'n_cells'

In [47]:
filtered_adata.write(cwd+'/outs/250102_kinker_anndata.h5ad')